# Importing Libraries

### This notebook will help you how to work with dataset. Enjoy

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
pio.renderers.default = 'iframe'

# Loading Dataset

In [3]:
df = pd.read_csv("/kaggle/input/nvidia-stocks-data-2025/NVDA.csv") 

# Exploratory Data Analysis

In [4]:
df.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000
1,1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000
2,1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000
3,1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000
4,1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6558 entries, 0 to 6557
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       6558 non-null   object 
 1   Adj Close  6558 non-null   float64
 2   Close      6558 non-null   float64
 3   High       6558 non-null   float64
 4   Low        6558 non-null   float64
 5   Open       6558 non-null   float64
 6   Volume     6558 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 358.8+ KB


In [6]:
df.isnull().sum()

Date         0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

There is no null value  in the dataset.

In [7]:
df["Date"]

0       1999-01-22
1       1999-01-25
2       1999-01-26
3       1999-01-27
4       1999-01-28
           ...    
6553    2025-02-10
6554    2025-02-11
6555    2025-02-12
6556    2025-02-13
6557    2025-02-14
Name: Date, Length: 6558, dtype: object

In [8]:
df['Date'] = pd.to_datetime(df['Date'])

df['Hour'] = df['Date'].dt.hour
df['Minute'] = df['Date'].dt.minute
df['Second'] = df['Date'].dt.second

df['UTC_Offset'] = df['Date'].dt.tz

# Separate day, month, and year
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [9]:
df.head()

,Date,Adj Close,Close,High,Low,Open,Volume,Hour,Minute,Second,UTC_Offset,Day,Month,Year
0,1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000,0,0,0,None,22,1,1999
1,1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000,0,0,0,None,25,1,1999
2,1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000,0,0,0,None,26,1,1999
3,1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000,0,0,0,None,27,1,1999
4,1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000,0,0,0,None,28,1,1999


In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Close'))

fig.update_layout(
    title="Closing Prices Over Time",
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    xaxis=dict(title='Date', color='white'),
    yaxis=dict(title='Price', color='white')
)

fig.show()

In [11]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['Volume'],
    nbinsx=30,  
    name='Volume',
    marker=dict(color='rgba(255, 0, 0, 0.7)'),  
    opacity=0.7
))

fig.update_layout(
    title="Volume Distribution",
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    xaxis=dict(title='Volume', color='white'),
    yaxis=dict(title='Frequency', color='white')
)

fig.show()

In [12]:
df['50D_MA'] = df['Close'].rolling(window=50).mean()
df['200D_MA'] = df['Close'].rolling(window=200).mean()


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Close'], mode='lines', name='Close Price', line=dict(color='cyan')
))

fig.add_trace(go.Scatter(
    x=df['Date'], y=df['50D_MA'], mode='lines', name='50-Day MA', line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=df['Date'], y=df['200D_MA'], mode='lines', name='200-Day MA', line=dict(color='magenta')
))

fig.update_layout(
    title="Close Price with 50 and 200-Day Moving Averages",
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    xaxis=dict(title='Date', color='white'),
    yaxis=dict(title='Price', color='white')
)

fig.show()


In [13]:
df['Daily_Return'] = df['Close'].pct_change() * 100
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['Daily_Return'].dropna(),
    name='Daily Return',
    marker=dict(color='rgba(0, 255, 0, 0.7)'), 
    opacity=0.7
))

fig.update_layout(
    title="Daily Return Distribution",
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    xaxis=dict(title='Daily Return (%)', color='white'),
    yaxis=dict(title='Frequency', color='white')
)

fig.show()

In [14]:
correlation_matrix = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].corr()

fig = px.imshow(correlation_matrix, 
                color_continuous_scale='RdBu', 
                title="Correlation Between Columns",
                labels={'color': 'Correlation'},
                color_continuous_midpoint=0)

fig.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    title_font=dict(color='white'),
    xaxis=dict(color='white'),
    yaxis=dict(color='white')
)

fig.show()

# ThankYou